# Recurrent Neural Networks

We have seen that convolutional neural networks allow us to efficiently model local structure in our data, but certainly this is not the only sort of structure that exists in nature. What else is there?

Consider the process of reading a document and understanding what it means. Locality is certainly still relevant, words close to each other are certainly more likely to relate to each other. But the relationships are not limited to any fixed size. A novel might mention some characteristic of the protagonist on the first page, and not refer to that characteristic again until the very last chapter. A convolution, limited as it is to a predefined size, does not seem like a good fit for modeling potentially infinitely long relationships. Clearly its possible however, after all, humans are known to use information aquired decades in the past to inform their current decision making. What sort of model could allow us to capture relationships between inputs separated by such large distances?

One option is a recurrent neural network. In its simplest form, a recurrent neural network is simply a single dense layer which is applied repeatedly to a sequence of inputs, and uses as an additional input, its output from the previous step in the sequence. In theory, such a model can capture relationships between inputs separated by an infinite distance. 

![Images](Images/rnn_loop.gif)

As with convolutional layers, it is often helpful to follow a recurrent layer with some sort of aggregation operation. Here, however, there is one additional option. We could just use the RNN's output from the very last step of the sequence. Since an RNN is theoretically capable of remembering all relevant information from the sequence this works. As a practical matter however, RNN's tend to forget information from distant inputs so its generally useful to use the intermediate outputs as well if those are of relevance. These can be captured through mean or max pooling (as we saw with convolutional neural networks), or through attention mechanisms which use additional layers to weight each output before averaging. 

### Preparing the Data

We will prepare our data as we did with convolutional neural networks, by creating a sequence of vectors corresponding to the input words in our training narratives. We accomplish this with a combination of the Keras Tokenizer, which maps narrative to a sequence of numbers representing each word in the narrative, and then an Embedding layer which maps each index to a vector. This is equivalent to representing each input word with a 1-hot vector and then multiplying each of these 1-hot vectors by a Dense layer.

In [3]:
from cac_net.gpu_manager import set_cuda_visible_devices

set_cuda_visible_devices()

CUDA_VISIBLE_DEVICES set to 2


In [6]:
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.text import Tokenizer

import pandas as pd

# read in our training data
df_train = pd.read_hdf('Data/msha_2010-2011.h5')
# read in our validation data
df_valid = pd.read_hdf('Data/msha_2012.h5')

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['NARRATIVE'])
X_train_seq = tokenizer.texts_to_sequences(df_train['NARRATIVE'])
X_valid_seq = tokenizer.texts_to_sequences(df_valid['NARRATIVE'])

# keras only accepts a one-hot encoding of the training labels
# we do that here
label_encoder = LabelBinarizer().fit(df_train['INJ_BODY_PART'])
y_train = label_encoder.transform(df_train['INJ_BODY_PART'])
y_valid = label_encoder.transform(df_valid['INJ_BODY_PART'])
n_codes = len(label_encoder.classes_)

In [5]:
print(X_train_seq[0])

[244, 29, 7152, 1570, 764, 213, 970, 4, 3198, 139, 5, 1924, 424, 223, 610, 1, 764, 29, 10, 1, 1570, 9, 3, 64, 2, 490, 110, 5, 213, 1, 764, 813, 4, 164, 317, 11, 6, 15, 54]


Although an RNN can, in theory, work with sequences of arbitrary length, for computational reasons it is necessary to make sure that each batch of input examples has the same length. We accomplsih this as we did with the convolutional neural networks, by padding (or truncating) all input sequences to the same length.

In [7]:
from keras.preprocessing import sequence

X_train_seq = sequence.pad_sequences(X_train_seq, maxlen=200)
X_valid_seq = sequence.pad_sequences(X_valid_seq, maxlen=200)

print(X_train_seq[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0  244   29 7152 1570  764  213  970
    4 3198  139    5 1924  424  223  610    1  764   29   10    1 1570
    9    3   64    2  490  110    5  213    1  764  813    4  164  317
   11 

We're now ready to specify the recurrent neural network. Here we use a particular type of recurrent neural network called an LSTM, which stands for Long-Short-Term-Memory. This contains a number of modifications which improve the performance. We will not go into the details in this class.

In [23]:
from keras.models import Model
from keras.layers import Dense, Input, Dropout
from keras.layers import Embedding, LSTM, GlobalMaxPooling1D, Concatenate
from keras.optimizers import Adam

input_text = Input(shape=(200,), dtype='int32')
embedding = Embedding(input_dim=len(tokenizer.word_index), 
                      output_dim=300, 
                      input_length=200)(input_text)
lstm = LSTM(units=256, 
            dropout=0.5, 
            recurrent_dropout=0.5, 
            return_sequences=True, )(embedding)
pool = GlobalMaxPooling1D()(lstm)
dropout = Dropout(0.5)(pool)
output = Dense(len(label_encoder.classes_), activation='softmax')(dropout)

model = Model(inputs=input_text, outputs=output)
optimer = Adam(lr=.001)
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [24]:
model.fit(x=X_train_seq, y=y_train,
          validation_data=(X_valid_seq, y_valid),
          batch_size=512, epochs=20)

Train on 18681 samples, validate on 9032 samples
Epoch 1/20
18681/18681 [==============================] - 12s 665us/step - loss: 3.3124 - acc: 0.1376 - val_loss: 3.0416 - val_acc: 0.1784
Epoch 2/20
18681/18681 [==============================] - 11s 598us/step - loss: 3.0917 - acc: 0.1781 - val_loss: 2.8815 - val_acc: 0.2005
Epoch 3/20
18681/18681 [==============================] - 11s 595us/step - loss: 2.6774 - acc: 0.3333 - val_loss: 2.1616 - val_acc: 0.4595
Epoch 4/20
18681/18681 [==============================] - 11s 591us/step - loss: 2.0518 - acc: 0.5030 - val_loss: 1.6645 - val_acc: 0.5858
Epoch 5/20
18681/18681 [==============================] - 11s 597us/step - loss: 1.6819 - acc: 0.5888 - val_loss: 1.4028 - val_acc: 0.6607
Epoch 6/20
18681/18681 [==============================] - 11s 596us/step - loss: 1.4313 - acc: 0.6610 - val_loss: 1.2002 - val_acc: 0.7184
Epoch 7/20
18681/18681 [==============================] - 11s 601us/step - loss: 1.2420 - acc: 0.7029 - val_loss: 1.0

A popular trick that sometimes results in some additional performance is to add another recurrent layer operating in the reverse direction of the sequence. We can implement this in Keras by wrapping our RNN with a Bidirectional layer. It simply creates two LSTM layers, one running left-to-right, the other right-to-left, and concatenates their outputs.

In [29]:
from keras.layers import Bidirectional


input_text = Input(shape=(200,), dtype='int32')
embedding = Embedding(input_dim=len(tokenizer.word_index), 
                      output_dim=300, 
                      input_length=200)(input_text)
lstm = Bidirectional(LSTM(units=128, 
                          dropout=0.5, 
                          recurrent_dropout=0.5, 
                          return_sequences=True),
                     merge_mode='concat')(embedding)
pool = GlobalMaxPooling1D()(lstm)
dropout = Dropout(0.5)(pool)
output = Dense(len(label_encoder.classes_), activation='softmax')(dropout)

model = Model(inputs=input_text, outputs=output)
optimer = Adam(lr=.001)
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [30]:
model.fit(x=X_train_seq, y=y_train,
          validation_data=(X_valid_seq, y_valid),
          batch_size=512, epochs=20)

Train on 18681 samples, validate on 9032 samples
Epoch 1/20
18681/18681 [==============================] - 24s 1ms/step - loss: 3.3387 - acc: 0.1560 - val_loss: 3.0156 - val_acc: 0.1784
Epoch 2/20
18681/18681 [==============================] - 21s 1ms/step - loss: 2.8840 - acc: 0.2558 - val_loss: 2.3971 - val_acc: 0.3813
Epoch 3/20
18681/18681 [==============================] - 21s 1ms/step - loss: 2.1882 - acc: 0.4535 - val_loss: 1.7683 - val_acc: 0.5531
Epoch 4/20
18681/18681 [==============================] - 21s 1ms/step - loss: 1.7133 - acc: 0.5761 - val_loss: 1.4413 - val_acc: 0.6332
Epoch 5/20
18681/18681 [==============================] - 21s 1ms/step - loss: 1.4412 - acc: 0.6438 - val_loss: 1.2146 - val_acc: 0.7088
Epoch 6/20
18681/18681 [==============================] - 21s 1ms/step - loss: 1.2241 - acc: 0.6992 - val_loss: 1.0530 - val_acc: 0.7356
Epoch 7/20
18681/18681 [==============================] - 21s 1ms/step - loss: 1.0593 - acc: 0.7361 - val_loss: 0.9456 - val_acc: